PRACTICA REALIZADA POR:

 * Manuel Cubertorer Gumbau
 * Francisco Javier Corrales Estrella
 
En la asignatura de Tipologia y ciclo de vida de los datos de Universidad Oberta Catalunya
Octubre 2021

**Evaluación inicial**

* ROBOTS.txt:
En primer lugar inspeccionamos el fichero robots.txt para ver la política sobre rastreos. El fichero no existe por lo que entendemos que el propietario del sitio web permite rastrear toda la página sin ningún tipo de restricción.

* Mapa del sitio web (sitemap.xml).

* Tamaño

* Tecnología usada


In [34]:

import builtwith
builtwith.parse('https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-23/1')


{'javascript-frameworks': ['AngularJS', 'Modernizr', 'jQuery'],
 'javascript-graphics': ['D3', 'Highcharts'],
 'font-scripts': ['Google Font API'],
 'web-frameworks': ['Twitter Bootstrap']}

Inicialmente, vamos a intentar inspeccionar la página usando BeatifulSoup, pero como hemos podido comprobar en el apartado anterior, la página tiene contenido en javascript y lo mas probable es que el contenido dinámico no lo podamos rastrear con esta técnica.

In [2]:
import requests

# Petición a la página web
#url principal
#TODO Navegar desde la pagina principal al dia de hoy
#url = "https://demanda.ree.es/visiona/peninsula/demanda/total"
url = "https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-23/1"
page = requests.get(url)

# Lectura del html del response
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content)


Efectivamente, el contenido del html que se carga mediante BeatifulSoup no es lo mismo que vemos explorando la web con las devtools de Chrome. Pasamos a utilizar otra técnica como Selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd

#opciones de navegacion
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = '/home/fjcorrales/Master Ciencia de datos/tipologia y ciclo de vida de los datos/PRACTICA-1/lib/chromedriver_linux64/chromedriver'

#Abrimos el navegador
driver = webdriver.Chrome(driver_path)

#Navegamos a la url
driver.get(url)

#Inicializamos los arrays para almacenar las variables
dias = list()
horas = list()
lectura_real = list()
lectura_prevista = list()
lectura_programada = list()

#Navegamos por los 30 dias anteriores
for i in range(5):            
    print("hoja",i)  
    # Leemos la información por horas    
    for t in range(2,146):
        #dia hora
        xpath_diahora = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[1]"
        element_diahora = driver.find_element_by_xpath(xpath_diahora)
        dia = element_diahora.text.split()[0]
        hora = element_diahora.text.split()[1]
        dias.append(dia)
        horas.append(hora)

        #Real
        xpath_real = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[2]"
        element_real = driver.find_element_by_xpath(xpath_real)
        lectura_real.append(element_real.text)

        #Prevista
        xpath_prevista = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[3]"
        element_prevista = driver.find_element_by_xpath(xpath_prevista)
        lectura_prevista.append(element_prevista.text)

        #Programada
        xpath_programada = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[4]"
        element_programada = driver.find_element_by_xpath(xpath_programada)
        lectura_programada.append(element_programada.text)

        #print(dia, hora, lectura_real, lectura_prevista,lectura_programada)  

    # Pulsamos click para navegar al dia anterior
    wait = WebDriverWait(driver, 5)
    elementButton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.md-icon-button main-menu md-button md-ink-ripple'.replace(' ','.'))))
    elementButton.click()


#Construimos el dataframe
df = pd.DataFrame({'Fecha': dias, 'Hora': horas, 'Lectura Real': lectura_real, 'Lectura_prevista': lectura_prevista, 'Lectura_programada': lectura_programada})

<ipython-input-5-f182a8709641>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


hoja 0


<ipython-input-5-f182a8709641>:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_diahora = driver.find_element_by_xpath(xpath_diahora)
<ipython-input-5-f182a8709641>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_real = driver.find_element_by_xpath(xpath_real)
<ipython-input-5-f182a8709641>:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_prevista = driver.find_element_by_xpath(xpath_prevista)
<ipython-input-5-f182a8709641>:53: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_programada = driver.find_element_by_xpath(xpath_programada)


hoja 1
hoja 2
hoja 3
hoja 4


In [6]:
df.head(10)

,Fecha,Hora,Lectura Real,Lectura_prevista,Lectura_programada
0,2021-10-22,21:00,29.690,29.744,29.024
1,2021-10-22,21:10,29.426,29.470,29.024
2,2021-10-22,21:20,29.186,29.171,29.024
3,2021-10-22,21:30,28.958,28.848,29.024
4,2021-10-22,21:40,28.416,28.501,29.024
5,2021-10-22,21:50,28.107,28.130,29.024
6,2021-10-22,22:00,27.253,27.733,26.637
7,2021-10-22,22:10,27.143,27.327,26.637
8,2021-10-22,22:20,26.598,26.922,26.637
9,2021-10-22,22:30,26.403,26.519,26.637
